## 🏛️🧩 Step 1 : Museum Data Modelling

### Import Libraries

In [1]:
# Import Libraries
import osmnx as ox # to fetch data from OpenStreetMap
import geopandas as gpd # to work with geospatial data
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

### Pull raw data for museums tag

In [2]:
tags = {
        "tourism": "museum"
       }

### Fetch Berlin Geometries

In [3]:
museum_raw = ox.features_from_place("Berlin, Germany", tags)


### Display basic info

In [4]:
print(f"Number of museum entries fetched: {len(museum_raw)}")
museum_raw.head()

Number of museum entries fetched: 245


geometry addr:housenumber  \
element id                                                      
node    73696610   POINT (13.36299 52.50761)            13-14   
        84644782   POINT (13.38221 52.50487)               90   
        259855486  POINT (13.43277 52.51048)                3   
        268591806  POINT (13.28894 52.45836)              NaN   
        281391655  POINT (13.48748 52.51451)              103   

                          addr:street                      email  fee  \
element id                                                              
node    73696610   Stauffenbergstraße  sekretariat@gdw-berlin.de   no   
        84644782     Stresemannstraße                        NaN   no   
        259855486        Koppenstraße      info@s-bahn-museum.de  yes   
        268591806                 NaN                        NaN  yes   
        281391655        Ruschestraße                        NaN  yes   

                                                               image  \
element id                                                             
node    73696610   http://commons.wikimedia.org/wiki/File:Bendele...   
        84644782                                                 NaN   
        259855486                                                NaN   
        268591806                                                NaN   
        281391655                                                NaN   

                                                                name  \
element id                                                             
node    73696610                   Gedenkstätte Deutscher Widerstand   
        84644782   Dokumentationszentrum Flucht, Vertreibung, Ver...   
        259855486                             Berliner S-Bahn-Museum   
        268591806                       Das Museum der Domäne Dahlem   
        281391655                                        Stasimuseum   

                                             name:en  \
element id                                             
node    73696610   Memorial to the German Resistance   
        84644782                                 NaN   
        259855486                                NaN   
        268591806                                NaN   
        281391655                       Stasi Museum   

                                               name:fr  \
element id                                               
node    73696610   Memorial de la Résistance Allemande   
        84644782                                   NaN   
        259855486                                  NaN   
        268591806                                  NaN   
        281391655                                  NaN   

                                            name:ru  ... name:tr nickname  \
element id                                           ...                    
node    73696610   Мемориал Немецкого Сопротивления  ...     NaN      NaN   
        84644782                                NaN  ...     NaN      NaN   
        259855486                               NaN  ...     NaN      NaN   
        268591806                               NaN  ...     NaN      NaN   
        281391655                               NaN  ...     NaN      NaN   

                  type alt_name:it construction_year photography name:ko  \
element id                                                                 
node    73696610   NaN         NaN               NaN         NaN     NaN   
        84644782   NaN         NaN               NaN         NaN     NaN   
        259855486  NaN         NaN               NaN         NaN     NaN   
        268591806  NaN         NaN               NaN         NaN     NaN   
        281391655  NaN         NaN               NaN         NaN     NaN   

                  changing_table:count changing_table:location  \
element id                                                       
node    73696610                   NaN               

### Save the raw data to a csv file

- Define file paths

In [5]:
raw_csv_path = "../sources/csv_files/museum_raw.csv"
raw_geojson_path = "../sources/geojson_files/museum_raw.geojson"

- Save csv & geojson to correct folders in sources

In [6]:
museum_raw.to_csv(raw_csv_path, index=False )

In [7]:
museum_raw.to_file(raw_geojson_path, driver="GeoJSON")

### Remove the columns with 75% or more missing data

In [8]:
# Step 1: Calculate percentage of missing values per column
missing_percent = museum_raw.isnull().mean() * 100

# Step 2: Identify columns with less than 75% missing
columns_to_keep = missing_percent[missing_percent < 75].index

# Step 3: Create a new filtered DataFrame
museum_filtered = museum_raw[columns_to_keep]

# Check shape and preview
print(museum_filtered.shape)
museum_filtered.head()

(245, 22)


geometry addr:housenumber  \
element id                                                      
node    73696610   POINT (13.36299 52.50761)            13-14   
        84644782   POINT (13.38221 52.50487)               90   
        259855486  POINT (13.43277 52.51048)                3   
        268591806  POINT (13.28894 52.45836)              NaN   
        281391655  POINT (13.48748 52.51451)              103   

                          addr:street  fee  \
element id                                   
node    73696610   Stauffenbergstraße   no   
        84644782     Stresemannstraße   no   
        259855486        Koppenstraße  yes   
        268591806                 NaN  yes   
        281391655        Ruschestraße  yes   

                                                                name  \
element id                                                             
node    73696610                   Gedenkstätte Deutscher Widerstand   
        84644782   Dokumentationszentrum Flucht, Vertreibung, Ver...   
        259855486                             Berliner S-Bahn-Museum   
        268591806                       Das Museum der Domäne Dahlem   
        281391655                                        Stasimuseum   

                                                       opening_hours tourism  \
element id                                                                     
node    73696610                Mo-Fr 09:00-18:00; Sa-Su 10:00-18:00  museum   
        84644782                           Tu-Su 10:00-19:00; Mo off  museum   
        259855486  We 12:00-16:00, Th,Fr 15:00-20:00, Su 14:00-18:00  museum   
        268591806                          Mo-Su 10:00-18:00; Th off  museum   
        281391655               Mo-Fr 10:00-18:00; Sa-Su 11:00-18:00  museum   

                                                          website wheelchair  \
element id                                                                     
node    73696610                       https://www.gdw-berlin.de/    limited   
        84644782   https://www.flucht-vertreibung-versoehnung.de/        yes   
        259855486                       https://s-bahn-museum.de/        yes   
        268591806             https://www.domaene-dahlem.de/home/    limited   
        281391655                                             NaN        yes   

                  wikidata  ... addr:postcode     addr:suburb  \
element id                  ...                                 
node    73696610   Q880798  ...           NaN             NaN   
        84644782       NaN  ...         10963       Kreuzberg   
        259855486      NaN  ...         10243  Friedrichshain   
        268591806      NaN  ...           NaN             NaN   
        281391655  Q570472  ...         10365     Lichtenberg   

                  toilets:wheelchair addr:country fee:icom_member    museum  \
element id                                                                    
node    73696610                 NaN          NaN             NaN       NaN   
        84644782                 yes          NaN             NaN       NaN   
        259855486                NaN           DE         unknown   railway   
        268591806                NaN          NaN              no  open_air   
        281391655                NaN           DE             yes   history   

                                       operator    contact:phone  \
element id                                                         
node    73696610                            NaN              NaN   
        84644782                            NaN              NaN   
        259855486  Berliner S-Bahn-Museum gGmbH              NaN   
        268591806                           NaN              NaN   
        281391655                    ASTAK e.V.  +49 30 55368 54   

                               contact:website building  
element id                                               
node    73696610           

### Check no Geometries missing

In [9]:
print("Missing geometries:", museum_filtered.geometry.isna().sum())

Missing geometries: 0


### Add Latitude & Longitude columns to filtered geodataframe

In [10]:
# Reproject and extract lat/lon
museum_filtered = museum_filtered.to_crs(epsg=4326)
museum_filtered['geometry'] = museum_filtered['geometry'].apply(
    lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point()
)
museum_filtered['latitude'] = museum_filtered.geometry.y
museum_filtered['longitude'] = museum_filtered.geometry.x

### Verify the lat/lon amounts are correct

In [11]:
print("Latitude range:", museum_filtered["latitude"].min(), "to", museum_filtered["latitude"].max())

print("Longitude range:", museum_filtered["longitude"].min(), "to", museum_filtered["longitude"].max())

Latitude range: 52.3878178 to 52.615457250000006
Longitude range: 13.13197421866466 to 13.6623573


### Reset index and drop redundant element column

In [12]:
museum_filtered = museum_filtered.reset_index()

# Drop the redundant column "element"
museum_filtered= museum_filtered.drop(columns=["element"],errors='ignore')

# Show final list of columns
print(museum_filtered.columns.tolist())

['id', 'geometry', 'addr:housenumber', 'addr:street', 'fee', 'name', 'opening_hours', 'tourism', 'website', 'wheelchair', 'wikidata', 'wikipedia', 'addr:city', 'addr:postcode', 'addr:suburb', 'toilets:wheelchair', 'addr:country', 'fee:icom_member', 'museum', 'operator', 'contact:phone', 'contact:website', 'building', 'latitude', 'longitude']


### Set the id to string

In [13]:
museum_filtered["id"] = museum_filtered["id"].astype(str)

In [14]:
museum_filtered.info(verbose=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   id                  245 non-null    object  
 1   geometry            245 non-null    geometry
 2   addr:housenumber    174 non-null    object  
 3   addr:street         180 non-null    object  
 4   fee                 180 non-null    object  
 5   name                243 non-null    object  
 6   opening_hours       193 non-null    object  
 7   tourism             245 non-null    object  
 8   website             125 non-null    object  
 9   wheelchair          157 non-null    object  
 10  wikidata            143 non-null    object  
 11  wikipedia           95 non-null     object  
 12  addr:city           167 non-null    object  
 13  addr:postcode       174 non-null    object  
 14  addr:suburb         129 non-null    object  
 15  toilets:wheelchair  64 non-null 

---

## 🏛️🔄 Step 2 : Museum Data Transformation

### Drop unnecessary columns
    - Dont need Berlin and DE as user knows they are looking for data in Berlin, Germany
    - Drop tourism column as all Museum
    - Drop suburb as it is the same as neighborhood which will be added later to make sure nothing is missing

In [15]:
museum_filtered.drop(columns=['addr:city', 'addr:country', 'tourism', 'addr:suburb'], errors='ignore', inplace=True)


### Copy the filtered database for record purposes and then standardise column names

In [16]:
museum_cleaned = museum_filtered.copy()

def clean_column(col):
    col = col.strip().lower().replace('addr:', '')
    col = col.replace(' ', '_')
    col = re.sub(r'[^a-z0-9_]', '', col)
    return col

museum_cleaned.columns = [clean_column(col) for col in museum_cleaned.columns]

### Rename columns for clarity

In [17]:
museum_cleaned.rename(columns={
    'postcode': 'postal_code',
    'housenumber': 'house_number',
    'toiletswheelchair': 'toilets_wheelchair_accessible',
    'feeicom_member': 'fee_icom_member',
    'contactphone': 'phone',
    'contactwebsite': 'contact_website',
    'museum': 'museum_type'
}, inplace=True)

In [18]:
museum_cleaned.columns.tolist()

['id',
 'geometry',
 'house_number',
 'street',
 'fee',
 'name',
 'opening_hours',
 'website',
 'wheelchair',
 'wikidata',
 'wikipedia',
 'postal_code',
 'toilets_wheelchair_accessible',
 'fee_icom_member',
 'museum_type',
 'operator',
 'phone',
 'contact_website',
 'building',
 'latitude',
 'longitude']

### Check the unique values

In [19]:
print("Unique fee:"  + str(museum_cleaned.fee.unique()))                                                        
print("Unique fee icom member:"  + str(museum_cleaned.fee_icom_member.unique()))                                                        
print("Unique wheelchair:"  + str(museum_cleaned.wheelchair.unique()))                                         
print("Unique toilets wheelchair accessible:"  + str(museum_cleaned.toilets_wheelchair_accessible.unique()))    
print("Unique museum_type:"  + str(museum_cleaned.museum_type.unique()))  
print("Unique 'building':"  + str(museum_cleaned['building'].unique())) 


Unique fee:['no' 'yes' nan]
Unique fee icom member:[nan 'unknown' 'no' 'yes' 'discount']
Unique wheelchair:['limited' 'yes' 'no' nan]
Unique toilets wheelchair accessible:[nan 'yes' 'no']
Unique museum_type:[nan 'railway' 'open_air' 'history' 'museum' 'local' 'nature' 'computer'
 'person' 'wax' 'art' 'children' 'living_history' 'archaeological'
 'technology' 'art,history,nature' 'science']
Unique 'building':[nan 'museum' 'civic' 'yes' 'church' 'public' 'apartments' 'commercial'
 'residential' 'bunker' 'industrial' 'government' 'detached' 'container']


### Fetch Districts & Neighbourhoods
- Load official Berlin districts GeoDataFrame

In [20]:
districts_gdf = gpd.read_file("../sources/geojson_files/lor_ortsteile.geojson")

In [21]:
districts_gdf.head(2)

,gml_id,spatial_name,spatial_alias,spatial_type,OTEIL,BEZIRK,FLAECHE_HA,geometry
0,re_ortsteil.0101,0101,Mitte,Polygon,Mitte,Mitte,1063.8748,"POLYGON ((13.41649 52.52696, 13.41635 52.52702..."
1,re_ortsteil.0102,0102,Moabit,Polygon,Moabit,Mitte,768.7909,"POLYGON ((13.33884 52.51974, 13.33884 52.51974..."


### Reproject GeoDataFrames to EPSG:4326 

In [22]:
museum_cleaned = museum_cleaned.to_crs(epsg=4326)
districts_gdf = districts_gdf.to_crs(epsg=4326)

### Spatial join with District Name and Neighbourhood_id(spatial_name)

In [23]:
museum_df_district = gpd.sjoin(
    museum_cleaned,
    districts_gdf[["BEZIRK", "spatial_name","geometry"]],
    how="left",
    predicate="within"
)

### Rename columns for clarity

In [24]:
museum_df_district = museum_df_district.rename(columns={
    "BEZIRK": "district",
    "spatial_name": "neighborhood_id"
}).drop(columns=["index_right"])  # drop district_number if not needed

### District mapping (official codes as strings)

In [25]:
district_mapping = {
    'Mitte': '11001001',
    'Friedrichshain-Kreuzberg': '11002002',
    'Pankow': '11003003',
    'Charlottenburg-Wilmersdorf': '11004004',
    'Spandau': '11005005',
    'Steglitz-Zehlendorf': '11006006',
    'Tempelhof-Schöneberg': '11007007',
    'Neukölln': '11008008',
    'Treptow-Köpenick': '11009009',
    'Marzahn-Hellersdorf': '11010010',
    'Lichtenberg': '11011011',
    'Reinickendorf': '11012012'
}

# Apply mapping to create district_id column (string)
museum_df_district['district_id'] = museum_df_district['district'].map(district_mapping).astype(str)

In [26]:
museum_df_district.head(2)

,id,geometry,house_number,street,fee,name,opening_hours,website,wheelchair,wikidata,...,museum_type,operator,phone,contact_website,building,latitude,longitude,district,neighborhood_id,district_id
0,73696610,POINT (13.36299 52.50761),13-14,Stauffenbergstraße,no,Gedenkstätte Deutscher Widerstand,Mo-Fr 09:00-18:00; Sa-Su 10:00-18:00,https://www.gdw-berlin.de/,limited,Q880798,...,NaN,NaN,NaN,NaN,NaN,52.507605,13.362986,Mitte,0104,11001001
1,84644782,POINT (13.38221 52.50487),90,Stresemannstraße,no,"Dokumentationszentrum Flucht, Vertreibung, Ver...",Tu-Su 10:00-19:00; Mo off,https://www.flucht-vertreibung-versoehnung.de/,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,52.504871,13.382214,Friedrichshain-Kreuzberg,0202,11002002


### Used reverse Geocoding and Nominatim to get Postal code & Street info
    - Created new columns at end to check against nulls in original columns

In [27]:
tqdm.pandas()

# Initialize geocoder
geolocator = Nominatim(user_agent="museum_locator")

# Define function to extract postal code and street
def get_postcode_and_street(row):
    try:
        location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
        address = location.raw.get('address', {})
        postcode = address.get('postcode')
        street = address.get('road') or address.get('pedestrian') or address.get('footway') or address.get('street')
        return pd.Series([postcode, street])
    except:
        return pd.Series([None, None])

# Apply to your DataFrame
museum_df_district[['postal_code_from_geo', 'street_from_geo']] = museum_df_district.progress_apply(get_postcode_and_street, axis=1)


100%|██████████| 245/245 [09:33<00:00,  2.34s/it]


In [28]:
museum_full = museum_df_district.copy()

### Check null count before replacing Nulls in postal code

In [29]:
print(museum_full[['postal_code']].isnull().sum())
print(museum_full[['street']].isnull().sum())

postal_code    71
dtype: int64
street    65
dtype: int64


### Replace the Nulls in the original columns

In [30]:
museum_full['postal_code'] = museum_full['postal_code'].fillna(museum_full['postal_code_from_geo'])
museum_full['street'] = museum_full['street'].fillna(museum_full['street_from_geo'])

### Check null values now to see if a difference

In [31]:
print(museum_full[['postal_code']].isnull().sum())
print(museum_full[['street']].isnull().sum())

postal_code    69
dtype: int64
street    64
dtype: int64


### Replace empty strings with NaN

In [32]:
museum_full.replace('', np.nan, inplace=True)

museum_full.head(2)

,id,geometry,house_number,street,fee,name,opening_hours,website,wheelchair,wikidata,...,phone,contact_website,building,latitude,longitude,district,neighborhood_id,district_id,postal_code_from_geo,street_from_geo
0,73696610,POINT (13.36299 52.50761),13-14,Stauffenbergstraße,no,Gedenkstätte Deutscher Widerstand,Mo-Fr 09:00-18:00; Sa-Su 10:00-18:00,https://www.gdw-berlin.de/,limited,Q880798,...,NaN,NaN,NaN,52.507605,13.362986,Mitte,0104,11001001,10785,Stauffenbergstraße
1,84644782,POINT (13.38221 52.50487),90,Stresemannstraße,no,"Dokumentationszentrum Flucht, Vertreibung, Ver...",Tu-Su 10:00-19:00; Mo off,https://www.flucht-vertreibung-versoehnung.de/,yes,NaN,...,NaN,NaN,NaN,52.504871,13.382214,Friedrichshain-Kreuzberg,0202,11002002,None,None


### Replace yes/no with True/False in fee column

In [33]:
museum_full["fee"] =museum_full["fee"].map({"yes": True, "no": False})

### Check if website NaN then use data from contact_website if available

In [34]:
museum_full['website'] = museum_full['website'].fillna(museum_full['contact_website'])

### Normalize the street name column

In [35]:
def normalize_street_name(name):
    if pd.isna(name):
        return np.nan
    # Replace underscores with spaces
    name = name.replace('_', ' ').replace('-', ' ')
    # Replace 'str.' or 'str' at end with ' Straße'
    name = re.sub(r'\bstr\.?\s*$', ' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'straße' if missing
    name = re.sub(r'(?<!\s)(straße)$', r' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'allee' if missing
    name = re.sub(r'(?<!\s)(allee)$', r' Allee', name, flags=re.IGNORECASE)
    # Ensure space before 'damm' if missing
    name = re.sub(r'(?<!\s)(damm)$', r' Damm', name, flags=re.IGNORECASE)
    # Ensure space before 'weg' if missing
    name = re.sub(r'(?<!\s)(weg)$', r' Weg', name, flags=re.IGNORECASE)
    # Ensure space before 'graben' if missing
    name = re.sub(r'(?<!\s)(graben)$', r' Graben', name, flags=re.IGNORECASE)
    # Ensure space before 'ufer' if missing
    name = re.sub(r'(?<!\s)(ufer)$', r' Ufer', name, flags=re.IGNORECASE)
    # Ensure space before 'korso' if missing
    name = re.sub(r'(?<!\s)(korso)$', r' Korso', name, flags=re.IGNORECASE)
    # Ensure space before 'zeile' if missing
    name = re.sub(r'(?<!\s)(zeile)$', r' Ziele', name, flags=re.IGNORECASE)
    # Ensure space before 'promenade' if missing
    name = re.sub(r'(?<!\s)(promenade)$', r' Promenade', name, flags=re.IGNORECASE)
    # Ensure space before 'kiez' if missing
    name = re.sub(r'(?<!\s)(kiez)$', r' Kiez', name, flags=re.IGNORECASE)
    # Ensure space before 'platz' if missing
    name = re.sub(r'(?<!\s)(platz)$', r' Platz', name, flags=re.IGNORECASE)
    # Ensure space before 'steig' if missing
    name = re.sub(r'(?<!\s)(steig)$', r' Steig', name, flags=re.IGNORECASE)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [36]:
museum_full['street'] = museum_full['street'].apply(normalize_street_name)

### Check and Normalize the telephone numbers

In [37]:
museum_full[museum_full['phone'].notna()]['phone'].head()

4      +49 30 55368 54
9     +49 30 200090300
16     +49-30-49910517
18     +49 30 20938591
20      +49 30 2291760
Name: phone, dtype: object

In [38]:
import pandas as pd
import numpy as np
import re

def normalize_phone_number(phone):
    if pd.isna(phone):
        return np.nan

    # Remove all non-digit and plus characters except leading '+'
    phone = re.sub(r'[^\d+]', '', phone)

    # Ensure it starts with +49
    if phone.startswith('0049'):
        phone = '+49' + phone[4:]
    elif phone.startswith('49'):
        phone = '+49' + phone[2:]
    elif not phone.startswith('+49'):
        phone = '+49' + phone.lstrip('0')

    # Insert space after country code and area code if it's Berlin (30)
    phone = re.sub(r'^\+49\s?30', '+49 30 ', phone)

    # Remove any remaining hyphens or double spaces
    phone = re.sub(r'\s+', ' ', phone).strip()

    return phone

In [39]:
museum_full['normalized_phone'] = museum_full['phone'].apply(normalize_phone_number)

In [40]:
museum_full[museum_full[['phone', 'normalized_phone']].notna().all(axis=1)][['phone', 'normalized_phone']].head()

,phone,normalized_phone
4,+49 30 55368 54,+49 30 5536854
9,+49 30 200090300,+49 30 200090300
16,+49-30-49910517,+49 30 49910517
18,+49 30 20938591,+49 30 20938591
20,+49 30 2291760,+49 30 2291760


### Drop unnecessary columns

In [41]:
museum_full.drop(columns=['geometry', 'district', 'postal_code_from_geo', 'street_from_geo', 'wikidata', 'fee_icon_member', 'contact_website', 'phone'], errors='ignore', inplace=True)

### Convert all text in columns to lowercase to avoid any duplications

In [42]:
text_cols = ['name', 'street', 'website', 'opening_hours', 'wheelchair', 'wikipedia', 'toilets_wheelchair_accessible', 'museum_type', 'building', 'operator']

for col in text_cols:
    if col in museum_full.columns:
        museum_full[col] = museum_full[col].apply(
            lambda x: x.strip().lower() if isinstance(x, str) else x
        )

### Check column data types**
    - All have correct type allocated so no need to change anything

In [43]:
museum_full.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 245 entries, 0 to 244
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             245 non-null    object 
 1   house_number                   174 non-null    object 
 2   street                         181 non-null    object 
 3   fee                            180 non-null    object 
 4   name                           243 non-null    object 
 5   opening_hours                  193 non-null    object 
 6   website                        204 non-null    object 
 7   wheelchair                     157 non-null    object 
 8   wikipedia                      95 non-null     object 
 9   postal_code                    176 non-null    object 
 10  toilets_wheelchair_accessible  64 non-null     object 
 11  fee_icom_member                69 non-null     object 
 12  museum_type                    72 non-null     

### Remove duplicates
    - In this case we have no duplicates but will keep this check for future runs incase

- See how many duplicate rows exist

In [44]:
museum_full.duplicated().sum()

np.int64(0)

- Display the actual duplicate rows

In [45]:
museum_full[museum_full.duplicated()]

,id,house_number,street,fee,name,opening_hours,website,wheelchair,wikipedia,postal_code,toilets_wheelchair_accessible,fee_icom_member,museum_type,operator,building,latitude,longitude,neighborhood_id,district_id,normalized_phone


- Remove duplicate rows and reset index

In [46]:
museum_full = museum_full.drop_duplicates().reset_index(drop=True)

### Remove row if name missing

In [47]:
museum_full = museum_full.dropna(subset=['name'])

### Reorder column names to be clearer

In [48]:
museum_listings = museum_full[['id', 'name', 'house_number', 'street',  'neighborhood_id', 'district_id', 'postal_code', 'website', 'normalized_phone', 'museum_type',
                               'operator', 'building', 'wikipedia', 'opening_hours', 'wheelchair', 'toilets_wheelchair_accessible', 'fee', 'latitude', 'longitude']]

In [49]:
museum_listings.to_csv("../sources/csv_files/museum_listings.csv", index=False)

### Final Summary of cleaned and Transformed Data

In [50]:
print(f"Number of rows: {museum_listings.shape[0]}")
print(f"Number of columns: {museum_listings.shape[1]}")

Number of rows: 243
Number of columns: 19


In [51]:
print("\nRemaining columns:")
print(museum_listings.columns.tolist())


Remaining columns:
['id', 'name', 'house_number', 'street', 'neighborhood_id', 'district_id', 'postal_code', 'website', 'normalized_phone', 'museum_type', 'operator', 'building', 'wikipedia', 'opening_hours', 'wheelchair', 'toilets_wheelchair_accessible', 'fee', 'latitude', 'longitude']


In [52]:
missing = museum_listings.isnull().sum()
print("\nMissing values after cleaning and transforming :")
print(missing)


Missing values after cleaning and transforming :
id                                 0
name                               0
house_number                      70
street                            63
neighborhood_id                    0
district_id                        0
postal_code                       68
website                           39
normalized_phone                 180
museum_type                      172
operator                         169
building                         172
wikipedia                        148
opening_hours                     51
wheelchair                        86
toilets_wheelchair_accessible    179
fee                               63
latitude                           0
longitude                          0
dtype: int64


In [53]:
missing_percent = museum_listings.isnull().mean() * 100
print(missing_percent.sort_values(ascending=False))

normalized_phone                 74.074074
toilets_wheelchair_accessible    73.662551
museum_type                      70.781893
building                         70.781893
operator                         69.547325
wikipedia                        60.905350
wheelchair                       35.390947
house_number                     28.806584
postal_code                      27.983539
street                           25.925926
fee                              25.925926
opening_hours                    20.987654
website                          16.049383
latitude                          0.000000
id                                0.000000
name                              0.000000
district_id                       0.000000
neighborhood_id                   0.000000
longitude                         0.000000
dtype: float64


In [54]:
museum_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 0 to 244
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             243 non-null    object 
 1   name                           243 non-null    object 
 2   house_number                   173 non-null    object 
 3   street                         180 non-null    object 
 4   neighborhood_id                243 non-null    object 
 5   district_id                    243 non-null    object 
 6   postal_code                    175 non-null    object 
 7   website                        204 non-null    object 
 8   normalized_phone               63 non-null     object 
 9   museum_type                    71 non-null     object 
 10  operator                       74 non-null     object 
 11  building                       71 non-null     object 
 12  wikipedia                      95 non-null     object 


In [55]:
museum_listings.head()

,id,name,house_number,street,neighborhood_id,district_id,postal_code,website,normalized_phone,museum_type,operator,building,wikipedia,opening_hours,wheelchair,toilets_wheelchair_accessible,fee,latitude,longitude
0,73696610,gedenkstätte deutscher widerstand,13-14,stauffenberg straße,0104,11001001,10785,https://www.gdw-berlin.de/,NaN,NaN,NaN,NaN,de:gedenkstätte deutscher widerstand,mo-fr 09:00-18:00; sa-su 10:00-18:00,limited,NaN,False,52.507605,13.362986
1,84644782,"dokumentationszentrum flucht, vertreibung, ver...",90,stresemann straße,0202,11002002,10963,https://www.flucht-vertreibung-versoehnung.de/,NaN,NaN,NaN,NaN,"de:stiftung flucht, vertreibung, versöhnung",tu-su 10:00-19:00; mo off,yes,yes,False,52.504871,13.382214
2,259855486,berliner s-bahn-museum,3,koppen straße,0201,11002002,10243,https://s-bahn-museum.de/,NaN,railway,berliner s-bahn-museum ggmbh,NaN,NaN,"we 12:00-16:00, th,fr 15:00-20:00, su 14:00-18:00",yes,NaN,True,52.510475,13.432765
3,268591806,das museum der domäne dahlem,NaN,NaN,0605,11006006,None,https://www.domaene-dahlem.de/home/,NaN,open_air,NaN,NaN,NaN,mo-su 10:00-18:00; th off,limited,NaN,True,52.458357,13.288944
4,281391655,stasimuseum,103,rusche straße,1103,11011011,10365,https://www.stasimuseum.de/,+49 30 5536854,history,astak e.v.,NaN,de:forschungs- und gedenkstätte normannenstraße,mo-fr 10:00-18:00; sa-su 11:00-18:00,yes,NaN,True,52.514507,13.487485


## 🏛️🗃️ Step 3: Populate Database

### Import extra Libraries

In [56]:
import psycopg2
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

### Credentials

In [ ]:
user_name=''
password=''

### Create the connection

In [58]:
# Conection
host = ''
port = '
database = ''
schema=''

#connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

### Use table tested above with contraints

In [59]:
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.museums (
    id VARCHAR(20) PRIMARY KEY,   -- OSM element id, stored as string
    name VARCHAR(200) NOT NULL,
    house_number VARCHAR(20),           
    street VARCHAR(200),
    neighborhood_id VARCHAR(20),
    district_id VARCHAR(200) NOT NULL,
    postal_code VARCHAR(20),
    website VARCHAR(250),
    normalized_phone VARCHAR(200),
    museum_type VARCHAR(200),   
    operator VARCHAR(200),
    building VARCHAR(200),           
    wikipedia VARCHAR(250),
    opening_hours VARCHAR(200),
    wheelchair VARCHAR(50),
    toilets_wheelchair_accessible VARCHAR(50),
    fee VARCHAR(50),
    latitude DECIMAL(9,6) NOT NULL,
    longitude DECIMAL(9,6) NOT NULL,
    CONSTRAINT district_id_fk FOREIGN KEY (district_id)
        REFERENCES berlin_source_data.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

### Execute the create table query

In [60]:
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction
    print("Table 'museums' created or already exists.")  

Table 'museums' created or already exists.


### Write the table to the layered Database using to.sql()

In [61]:
museum_listings.to_sql(
    'museums',      
    engine,
    schema=schema,
    if_exists='append', # ✅ keeps table, just inserts data
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

DataFrame sent to PostgreSQL using .to_sql() with psycopg2!


### Query the table with SQL

In [62]:
query = f"""
SELECT * 
FROM berlin_source_data.museums
LIMIT 10;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,id,name,house_number,street,neighborhood_id,district_id,postal_code,website,normalized_phone,museum_type,operator,building,wikipedia,opening_hours,wheelchair,toilets_wheelchair_accessible,fee,latitude,longitude
0,73696610,gedenkstätte deutscher widerstand,13-14,stauffenberg straße,0104,11001001,10785,https://www.gdw-berlin.de/,None,None,None,None,de:gedenkstätte deutscher widerstand,mo-fr 09:00-18:00; sa-su 10:00-18:00,limited,None,false,52.507605,13.362986
1,84644782,"dokumentationszentrum flucht, vertreibung, ver...",90,stresemann straße,0202,11002002,10963,https://www.flucht-vertreibung-versoehnung.de/,None,None,None,None,"de:stiftung flucht, vertreibung, versöhnung",tu-su 10:00-19:00; mo off,yes,yes,false,52.504871,13.382214
2,259855486,berliner s-bahn-museum,3,koppen straße,0201,11002002,10243,https://s-bahn-museum.de/,None,railway,berliner s-bahn-museum ggmbh,None,None,"we 12:00-16:00, th,fr 15:00-20:00, su 14:00-18:00",yes,None,true,52.510475,13.432765
3,268591806,das museum der domäne dahlem,None,None,0605,11006006,None,https://www.domaene-dahlem.de/home/,None,open_air,None,None,None,mo-su 10:00-18:00; th off,limited,None,true,52.458357,13.288944
4,281391655,stasimuseum,103,rusche straße,1103,11011011,10365,https://www.stasimuseum.de/,+49 30 5536854,history,astak e.v.,None,de:forschungs- und gedenkstätte normannenstraße,mo-fr 10:00-18:00; sa-su 11:00-18:00,yes,None,true,52.514507,13.487485
5,292700476,feuerwehrmuseum berlin,5,veit straße,1202,11012012,13507,https://www.feuerwehrmuseum-berlin.de/,None,history,None,None,de:feuerwehrmuseum berlin,"tu,th 09:00-16:00, we 09:00-19:00, fr,sa 10:00...",no,None,true,52.586307,13.286313
6,292738889,museum für kommunikation,16,leipziger straße,0101,11001001,10117,https://www.mfk-berlin.de/,None,None,None,None,de:museum für kommunikation berlin,tu 09:00-20:00; we-fr 09:00-17:00; sa-su 10:00...,yes,yes,false,52.509931,13.387255
7,297049901,heimatmuseum köpenick,None,None,0910,11009009,None,None,None,None,None,None,None,"tu,we 10:00-16:00; th 10:00-18:00; su 14:00-18:00",no,None,false,52.445364,13.579097
8,335860469,ethnologisches museum,None,im winkel,0605,11006006,14195,None,None,None,None,None,None,None,limited,yes,None,52.460600,13.292956
9,340388625,deutschlandmuseum,7,leipziger platz,0101,11001001,10117,https://www.deutschlandmuseum.de/,+49 30 200090300,history,betreibergesellschaft deutschland museum dm gmbh,None,de:deutschlandmuseum,mo-su 10:00-20:00,yes,no,true,52.508788,13.378434


### Check all districts exist

In [63]:
query = f"""
SELECT mu.district_id, di.district_id, COUNT(*) as museum_count
FROM berlin_source_data.museums mu
JOIN berlin_source_data.districts di
ON mu.district_id = di.district_id
GROUP BY mu.district_id, di.district_id
ORDER BY mu.district_id;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,district_id,district_id,museum_count
0,11001001,11001001,86
1,11002002,11002002,27
2,11003003,11003003,10
3,11004004,11004004,29
4,11005005,11005005,6
5,11006006,11006006,20
6,11007007,11007007,16
7,11008008,11008008,12
8,11009009,11009009,16
9,11010010,11010010,6
